In [1]:
import re

In [2]:
prefixes="@prefix case: <https://w3id.org/def/NyOn#Case> .\n@prefix nyon: <https://w3id.org/def/NyOn#> .\n@prefix rdfs:	<http://www.w3.org/2000/01/rdf-schema#> .\n@prefix foaf:	<http://xmlns.com/foaf/0.1/> .\n@prefix co: <https://w3id.org/def/NyOn#Court_Official> .\n@prefix schema: <https://schema.org/> .\n@prefix party: <https://w3id.org/def/NyOn#Party>	.\n@prefix court: <https://w3id.org/def/NyOn#Courts>	.\n@prefix decision: <https://w3id.org/def/NyOn#Court_Decision>	. \n@prefix juri:<https://w3id.org/def/NyOn#Jurisdiction> . \n"

In [3]:
print(prefixes)


@prefix case: <https://w3id.org/def/NyOn#Case> .
@prefix nyon: <https://w3id.org/def/NyOn#> .
@prefix rdfs:	<http://www.w3.org/2000/01/rdf-schema#> .
@prefix foaf:	<http://xmlns.com/foaf/0.1/> .
@prefix co: <https://w3id.org/def/NyOn#Court_Official> .
@prefix schema: <https://schema.org/> .
@prefix party: <https://w3id.org/def/NyOn#Party>	.
@prefix court: <https://w3id.org/def/NyOn#Courts>	.
@prefix decision: <https://w3id.org/def/NyOn#Court_Decision>	. 
@prefix juri:<https://w3id.org/def/NyOn#Jurisdiction> . 



In [4]:
with open('RE.txt') as f:
   lines = [line.rstrip() for line in f]

In [5]:
lines

[' 1397402.txt',
 'CASE hasCaseId 200111KS1SC',
 '200111KS1SC hasCaseName   GORDHAN DAS VS. PIRKHAN & ORS.  on 27/11/2001',
 '200111KS1SC hasParty PETITIONER',
 'PETITIONER hasName   GORDHAN DAS',
 '200111KS1SC hasParty RESPONDENT',
 'RESPONDENT hasName   PIRKHAN & ORS.',
 '200111KS1SC hasDate  27/11/2001',
 '200111KS1SC hasYear 2001',
 '200111KS1SC hasMonth 11',
 '200111KS1SC hasAppealNo  Appeal No.219 of 1975',
 '200111KS1SC hasCaseNo  CASE NO.: Appeal (civil) 4391 of 1990',
 '200111KS1SC hasCaseType  civil',
 '200111KS1SC hasCourtOfficial Judge',
 'Judge hasName   Syed Shah Mohammed Quadri',
 'Judge hasName   S.N. Phukan',
 '200111KS1SC hasCourt  High Court',
 ' High Court hasLocation  Madhya Pradesh',
 '200111KS1SC hasCourtDecision accepted',
 '',
 ' 156593612.txt',
 'CASE hasCaseId 199604KS1SC',
 '199604KS1SC hasCaseName   HEIRS OF VRAJLAL J. GANATRA VS. HAIRS OF PARSHOTTAM S. SHAH  on 30/04/1996',
 '199604KS1SC hasParty PETITIONER',
 'PETITIONER hasName   HEIRS OF VRAJLAL J. GANA

In [6]:

p=[]
r=[]
j=[]
p_and_r_and_j =0
for line in lines:
  if 'hasParty PETITIONER' in line:
    p_and_r_and_j=1
  if 'hasParty RESPONDENT' in line:
    p_and_r_and_j=2
  if 'hasCourtOfficial Judge' in line:
    p_and_r_and_j=3

  if 'hasName' in line:
    tmp= (line.split("hasName",1)[1]).strip()
    if p_and_r_and_j==1:
      if tmp not in p:
        p.append(tmp)
    if p_and_r_and_j==2:
      if tmp not in r:
        r.append(tmp)
    if p_and_r_and_j==3:
      if tmp not in j:
        j.append(tmp)


In [20]:
number_of_cases=0
author=[]
case_end=0
judges=[]
petetioner=''
respondent=''
data=""
casename=""
p_and_r_and_j =0
for line in lines:
  if '.txt' in line:
    number_of_cases+=1
    if case_end+1 != number_of_cases:

      if len(petetioner)!=0 and len(respondent)!=0:
        p_index= p.index(petetioner)   
      #if len(r)!=0:
        r_index= r.index(respondent)
        data+="nyon:hasParty party:"+re.sub('[^A-Za-z0-9_]+', '', p[p_index].replace(" ", "_"))+", party:"+re.sub('[^A-Za-z0-9_]+', '', r[r_index].replace(" ", "_"))+";\n"


      j_index= []
      for i in judges:
        j_index.append(j.index(i))
      text=[]
      temp=''
      
      if len(judges)!=0:
        for i in j_index:
          text.append("co:"+re.sub('[^A-Za-z0-9_]+',
                                   '', j[i].replace(" ", "_")))
        for i in text:
          temp+=i+","
        temp=temp[:-1]

        data+="nyon:hasCourtOfficial "+temp+";\n"
      data+=".\n"
      f=0
      #data+=":case"+str(number_of_cases-1)+'P1 a schema:Petitioner, foaf:Person ;\n \trdfs:hasName "'+petetioner+'";\n .\n'
      #data+=":case"+str(number_of_cases-1)+'P1 a schema:Respondent, foaf:Person ;\n \trdfs:hasName "'+respondent+'";\n .\n'

      case_end+=1
      p_and_r_and_j =0
      judges=[]
      petetioner=''
      respondent=''
  
  if 'hasCaseId' in line:
    
    casename= (line.split("hasCaseId ",1)[1]).strip()
    data+="case:"+casename+" a nyon:Case;\n"

  if 'hasCaseName' in line:
    
    case= (line.split("hasCaseName ",1)[1]).strip()    
    data+='nyon:hasCaseName  "'+case+'"@en;\n'

  if 'hasDate' in line:#to be done using loop 
    tmp= line.split("hasDate ",1)[1]
    data+='nyon:hasDate "'+tmp.strip()+'";\n'
  if 'hasYear' in line:#to be done using loop 
    tmp= line.split("hasYear",1)[1]
    data+='nyon:hasYear "'+tmp.strip()+'";\n'

  if 'hasMonth' in line:#to be done using loop 
    tmp= line.split("hasMonth",1)[1]
    data+='nyon:hasMonth "'+tmp.strip()+'";\n'
  if 'hasJurisdiction' in line:#to be done using loop 
    tmp= line.split("hasJurisdiction",1)[1]
    if 'appellate' in tmp.lower():
      data+='nyon:hasJurisdiction juri:Appellant_Jurisdiction;\n'


  if 'hasAppealNo' in line:
    tmp= line.split("hasAppealNo ",1)[1]
    data+='nyon:hasAppealNo "'+tmp.strip()+'";\n'

  if 'hasCaseNo' in line:
    tmp= line.split("hasCaseNo ",1)[1]
    data+='nyon:hasCaseNo "'+tmp.strip()+'";\n'
  
  if 'hasCourtDecision' in line:
    tmp= line.split("hasCourtDecision ",1)[1]
    if 'accepted' in tmp.lower():
      data+='nyon:hasCourtDecision decision:acc;\n'
    else:
      data+='nyon:hasCourtDecision decision:rej;\n'


  if 'hasCaseType' in line:
    tmp= line.split("hasCaseType ",1)[1]
    if 'civil' in tmp.lower():
      data+='nyon:hasCaseType nyon:Civil;\n'
    else:
      data+='nyon:hasCaseType nyon:Criminal;\n'

  
  if 'hasAuthor' in line:
    tmp= (line.split("hasAuthor ",1)[1]).strip()
    if tmp in j:
      index= j.index(tmp)
      author.append(tmp)
    else:
      index=len(j)
      j.append(tmp)
      
      author.append(tmp)
    print(len(j))
    print(index)
    data+='nyon:hasAuthor co:'+re.sub('[^A-Za-z0-9_]+', '', j[index].replace(" ", "_"))+';\n'
  
  if 'hasCourt' in line:
    if f==0:
      data+='nyon:hasCourt co:sc;\n'
      f=1
  
  """if 'hasLocation' in line:
    tmp= line.split("hasLocation ",1)[1]
    data+='nyon:hasLocation "'+tmp.strip()+'";\n'"""
  



  if 'hasParty PETITIONER' in line:
    p_and_r_and_j=1
  if 'hasParty RESPONDENT' in line:
    p_and_r_and_j=2
  if 'hasCourtOfficial Judge' in line:
    p_and_r_and_j=3

  if 'hasName' in line:
    tmp= line.split("hasName",1)[1]
    if p_and_r_and_j==1:
      petetioner+=tmp.strip()
    if p_and_r_and_j==2:
      respondent+=tmp.strip()      
    if p_and_r_and_j==3:
      judges.append(tmp.strip())


if len(petetioner)!=0 and len(respondent)!=0:
  p_index= p.index(petetioner)   
  r_index= r.index(respondent)
  data+="nyon:hasParty party:"+re.sub('[^A-Za-z0-9_]+', '', p[p_index].replace(" ", "_"))+", party:"+re.sub('[^A-Za-z0-9_]+', '', r[r_index].replace(" ", "_"))+";\n"


j_index= []
for i in judges:
  j_index.append(j.index(i))
text=[]
temp=''

if len(judges)!=0:
  for i in j_index:
    text.append("co:"+re.sub('[^A-Za-z0-9_]+', '', j[i].replace(" ", "_")))
  for i in text:
    temp+=i+","
  temp=temp[:-1]

  data+="nyon:hasCourtOfficial "+temp+";\n"
data+=".\n"


88
77
88
8
88
13
88
78
88
36
88
79
88
38
88
80
88
48
88
15
88
13
88
38
88
81
88
82
88
38
88
83
88
84
88
85
88
65
88
86
88
38
88
84
88
83
88
73
88
87


In [21]:
individual=""
for val in range(len(j)):
  if j[val] in author:
    individual+="co:"+re.sub('[^A-Za-z0-9_]+', '', j[val].replace(" ", "_"))+' rdfs:type co:Judge; \nrdfs:type nyon:Author; \nnyon:works_in court:sc; \na foaf:Person ;\n \trdfs:label "'+j[val]+'"@en .\n'
  else:
    individual+="co:"+re.sub('[^A-Za-z0-9_]+', '', j[val].replace(" ", "_"))+' rdfs:type co:Judge; \nnyon:works_in court:SupremeCourt; \na foaf:Person ;\n \trdfs:label "'+j[val]+'"@en .\n'
for val in range(len(p)):
  temp= p[val].lower()
  if 'state' in temp or 'govt' in temp:
    individual+="party:"+re.sub('[^A-Za-z0-9_]+', '', p[val].replace(" ", "_"))+' rdfs:type nyon:Petitioner; \nnyon:PartType schema:State; \na foaf:Person ;\n \trdfs:label "'+p[val]+'"@en .\n'
  elif 'pvt' in temp or 'corporation' in temp or 'company' in temp:
    individual+="party:"+re.sub('[^A-Za-z0-9_]+', '', p[val].replace(" ", "_"))+' rdfs:type nyon:Petitioner; \nnyon:PartType schema:Orgaization;\na foaf:Person ;\n \trdfs:label "'+p[val]+'"@en .\n'
  else:
    individual+="party:"+re.sub('[^A-Za-z0-9_]+', '', p[val].replace(" ", "_"))+' rdfs:type nyon:Petitioner; \na foaf:Person ;\n \trdfs:label "'+p[val]+'"@en .\n'

for val in range(len(r)):
  temp= r[val].lower()
  if 'state' in temp or 'govt' in temp:
    individual+="party:"+re.sub('[^A-Za-z0-9_]+', '', r[val].replace(" ", "_"))+' rdfs:type nyon:Petitioner; \nnyon:PartType schema:State; \na foaf:Person ;\n \trdfs:label "'+r[val]+'"@en .\n'
  elif 'pvt' in temp or 'corporation' in temp or 'company' in temp:
    individual+="party:"+re.sub('[^A-Za-z0-9_]+', '', r[val].replace(" ", "_"))+' rdfs:type nyon:Petitioner; \nnyon:PartType schema:Orgaization;\na foaf:Person ;\n \trdfs:label "'+r[val]+'"@en .\n'
  else:
    individual+="party:"+re.sub('[^A-Za-z0-9_]+', '', r[val].replace(" ", "_"))+' rdfs:type nyon:Respondent; \na foaf:Person ;\n \trdfs:label "'+r[val]+'"@en .\n'



In [22]:
court='court:sc rdfs:type court:Apex_Court; \n rdfs:label "Supreme Court of India"@en .\ndecision:acc rdfs:type decision:Judgement; \n rdfs:label "Accepted"@en .\ndecision:rej rdfs:type decision:Judgement; \n rdfs:label "Disposed"@en .\n juri:appellant  rdfs:type juri:Appellant_Jurisdiction; \n rdfs:label "Apppellant Jurisdiction"@en .'


In [23]:
print(prefixes)
print(court)
print(individual)
print(data)

@prefix case: <https://w3id.org/def/NyOn#Case> .
@prefix nyon: <https://w3id.org/def/NyOn#> .
@prefix rdfs:	<http://www.w3.org/2000/01/rdf-schema#> .
@prefix foaf:	<http://xmlns.com/foaf/0.1/> .
@prefix co: <https://w3id.org/def/NyOn#Court_Official> .
@prefix schema: <https://schema.org/> .
@prefix party: <https://w3id.org/def/NyOn#Party>	.
@prefix court: <https://w3id.org/def/NyOn#Courts>	.
@prefix decision: <https://w3id.org/def/NyOn#Court_Decision>	. 
@prefix juri:<https://w3id.org/def/NyOn#Jurisdiction> . 

court:sc rdfs:type court:Apex_Court; 
 rdfs:label "Supreme Court of India"@en .
decision:acc rdfs:type decision:Judgement; 
 rdfs:label "Accepted"@en .
decision:rej rdfs:type decision:Judgement; 
 rdfs:label "Disposed"@en .
 juri:appellant  rdfs:type juri:Appellant_Jurisdiction; 
 rdfs:label "Apppellant Jurisdiction"@en .
co:Syed_Shah_Mohammed_Quadri rdfs:type co:Judge; 
nyon:works_in court:SupremeCourt; 
a foaf:Person ;
 	rdfs:label "Syed Shah Mohammed Quadri"@en .
co:SN_Phukan

#try


In [ ]:
number_of_cases=0
case_end=0
judges=[]
petetioner=''
respondent=''
data=""
casename=""
p_and_r_and_j =0
for line in lines:
  if '.txt' in line:
    number_of_cases+=1
    if case_end+1 != number_of_cases:
      data+="dc:hasParty :case"+str(number_of_cases-1)+"P1, :case"+str(number_of_cases-1)+"P2;\n"
      text=[]
      temp=''
      if len(judges)!=0:
        for i in range(len(judges)):
          text.append(":case"+str(number_of_cases-1)+"Judge"+str(i+1))
        for i in text:
          temp+=i+","
        temp=temp[:-1]

        data+="dc:hasCourtOfficial "+temp+";\n"
      data+=".\n"
      data+=":case"+str(number_of_cases-1)+'P1 a schema:Petitioner, foaf:Person ;\n \trdfs:hasName "'+petetioner+'";\n .\n'
      data+=":case"+str(number_of_cases-1)+'P1 a schema:Respondent, foaf:Person ;\n \trdfs:hasName "'+respondent+'";\n .\n'

      if len(judges)>0:
        for i in range(len(judges)):
          data+= text[i]+' a schema:Judge, foaf:Person ;\n \trdfs:hasName "'+str(judges[i])+'";\n .\n'
      case_end+=1
      p_and_r_and_j =0
      judges=[]
      petetioner=''
      respondent=''
    data+=":case"+str(number_of_cases)+"\n"

  if 'hasCaseName' in line:
    casename= line.split("hasCaseName ",1)[1]
    temp= casename.replace(" ", "_")
    temp=re.sub('[^A-Za-z0-9_]+', '', temp)
    data+="a schema:"+str(temp.strip())+";\n"
    data+='rdfs:label "'+casename.strip()+'";\n'

  if 'hasDate' in line:#to be done using loop 
    tmp= line.split("hasDate ",1)[1]
    data+='rdfs:hasDate "'+tmp.strip()+'";\n'

  if 'hasAppealNo' in line:
    tmp= line.split("hasAppealNo ",1)[1]
    data+='rdfs:hasAppealNo "'+tmp.strip()+'";\n'

  if 'hasCaseNo' in line:
    tmp= line.split("hasCaseNo ",1)[1]
    data+='rdfs:hasCaseNo "'+tmp.strip()+'";\n'
  
  if 'hasCaseType' in line:
    tmp= line.split("hasCaseType ",1)[1]
    data+='rdfs:hasCaseType "'+tmp.lower()+'";\n'
  
  if 'hasAuthor' in line:
    tmp= line.split("hasAuthor ",1)[1]
    data+='rdfs:hasAuthor "'+tmp.strip()+'";\n'
  
  if 'hasCourt' in line:
    tmp= line.split("hasCourt",1)[1]
    data+='rdfs:hasCourt "'+tmp.strip()+'";\n'
  
  if 'hasLocation' in line:
    tmp= line.split("hasLocation ",1)[1]
    data+='rdfs:hasLocation "'+tmp.strip()+'";\n'



  if 'hasParty PETITIONER' in line:
    p_and_r_and_j=1
  if 'hasParty RESPONDENT' in line:
    p_and_r_and_j=2
  if 'hasCourtOfficial Judge' in line:
    p_and_r_and_j=3

  if 'hasName' in line:
    tmp= line.split("hasName ",1)[1]
    if p_and_r_and_j==1:
      petetioner+=tmp.strip()
    if p_and_r_and_j==2:
      respondent+=tmp.strip()      
    if p_and_r_and_j==3:
      judges.append(tmp.strip())

data+="dc:hasParty :case"+str(number_of_cases)+"P1, :case"+str(number_of_cases)+"P2;\n"
text=[]
temp=''
if len(judges)!=0:
  for i in range(len(judges)):
    text.append(":case"+str(number_of_cases)+"Judge"+str(i+1))
  for i in text:
    temp+=i+","
  temp=temp[:-1]

  data+="dc:hasCourtOfficial "+temp.strip()+";\n "
data+=".\n"
data+=":case"+str(number_of_cases)+'P1 a schema:Petitioner, foaf:Person ;\n \trdfs:hasName "'+petetioner+'";\n .\n'
data+=":case"+str(number_of_cases)+'P1 a schema:Respondent, foaf:Person ;\n \trdfs:hasName "'+respondent+'";\n .\n'

if len(judges)>0:
  for i in range(len(judges)):
    data+= text[i]+' a schema:Judge, foaf:Person ;\n \trdfs:hasName "'+str(judges[i])+'";\n .\n'
  


In [ ]:
scheme=":Case a schema:Thing ;\n dc:hasCaseName "
text=""
for i in range(number_of_cases):
  text+=":case"+str(i+1)+","
text=text[:-1]
scheme+=text+";\n .\n"

In [ ]:
print(prefixes)
print(scheme)
print(data)

In [ ]:
number_of_cases=0
case_end=0
judges=[]
petetioner=''
respondent=''
data=""
casename=""
p_and_r_and_j =0
for line in lines:
  if '.txt' in line:
    number_of_cases+=1
    if case_end+1 != number_of_cases:
      if len(petetioner)!=0 and len(respondent)!=0:
        p_index= p.index(petetioner)   
      #if len(r)!=0:
        r_index= r.index(respondent)
        data+="dc:hasParty :P"+str(p_index+1)+", :R"+str(r_index+1)+";\n"


      j_index= []
      for i in judges:
        j_index.append(j.index(i))
      text=[]
      temp=''
      
      if len(judges)!=0:
        for i in j_index:
          text.append(":J"+str(i+1))
        for i in text:
          temp+=i+","
        temp=temp[:-1]

        data+="dc:hasCourtOfficial "+temp+";\n"
      data+=".\n"
      #data+=":case"+str(number_of_cases-1)+'P1 a schema:Petitioner, foaf:Person ;\n \trdfs:hasName "'+petetioner+'";\n .\n'
      #data+=":case"+str(number_of_cases-1)+'P1 a schema:Respondent, foaf:Person ;\n \trdfs:hasName "'+respondent+'";\n .\n'

      case_end+=1
      p_and_r_and_j =0
      judges=[]
      petetioner=''
      respondent=''
    data+=":case"+str(number_of_cases)+"\n"

  if 'hasCaseName' in line:
    casename= line.split("hasCaseName ",1)[1]
    temp= casename.replace(" ", "_")
    temp=re.sub('[^A-Za-z0-9_]+', '', temp)
    data+="a schema:"+str(temp.strip())+";\n"
    data+='rdfs:label "'+casename.strip()+'";\n'

  if 'hasDate' in line:#to be done using loop 
    tmp= line.split("hasDate ",1)[1]
    data+='rdfs:hasDate "'+tmp.strip()+'";\n'

  if 'hasAppealNo' in line:
    tmp= line.split("hasAppealNo ",1)[1]
    data+='rdfs:hasAppealNo "'+tmp.strip()+'";\n'

  if 'hasCaseNo' in line:
    tmp= line.split("hasCaseNo ",1)[1]
    data+='rdfs:hasCaseNo "'+tmp.strip()+'";\n'
  
  if 'hasCaseType' in line:
    tmp= line.split("hasCaseType ",1)[1]
    data+='rdfs:hasCaseType "'+tmp.lower()+'";\n'
  
  if 'hasAuthor' in line:
    tmp= (line.split("hasAuthor ",1)[1]).strip()
    if tmp in j:
      index= j.index(tmp)
    else:
      index=len(j)
      j.append(tmp)
      
    data+='rdfs:hasAuthor :J'+str(index+1)+';\n'
  
  if 'hasCourt' in line:
    tmp= line.split("hasCourt",1)[1]
    data+='rdfs:hasCourt "'+tmp.strip()+'";\n'
  
  if 'hasLocation' in line:
    tmp= line.split("hasLocation ",1)[1]
    data+='rdfs:hasLocation "'+tmp.strip()+'";\n'



  if 'hasParty PETITIONER' in line:
    p_and_r_and_j=1
  if 'hasParty RESPONDENT' in line:
    p_and_r_and_j=2
  if 'hasCourtOfficial Judge' in line:
    p_and_r_and_j=3

  if 'hasName' in line:
    tmp= line.split("hasName ",1)[1]
    if p_and_r_and_j==1:
      petetioner+=tmp.strip()
    if p_and_r_and_j==2:
      respondent+=tmp.strip()      
    if p_and_r_and_j==3:
      judges.append(tmp.strip())



p_index= p.index(petetioner)      
r_index= r.index(respondent)
data+="dc:hasParty :P"+str(p_index+1)+", :R"+str(r_index+1)+";\n"


j_index= []
for i in judges:
  j_index.append(j.index(i))
print(j_index)
text=[]
temp=''

if len(judges)!=0:
  for i in j_index:
    print(i)
    text.append(":J"+str(i+1))
  for i in text:
    temp+=i+","
  temp=temp[:-1]

  data+="dc:hasCourtOfficial "+temp+";\n"
data+=".\n"

for val in range(len(p)):
  data+=":P"+str(val+1)+' a schema:Petitioner, foaf:Person ;\n \trdfs:hasName "'+p[val]+'";\n .\n'

for val in range(len(r)):
  data+=":R"+str(val+1)+' a schema:Respondent, foaf:Person ;\n \trdfs:hasName "'+r[val]+'";\n .\n'

for val in range(len(j)):
  data+=":J"+str(val+1)+' a schema:Judge, foaf:Person ;\n \trdfs:hasName "'+j[val]+'";\n .\n'
  


[8, 9]
8
9


In [ ]:
scheme=":Case a schema:Thing ;\n dc:hasCaseName "
text=""
for i in range(number_of_cases):
  text+=":case"+str(i+1)+","
text=text[:-1]
scheme+=text+";\n .\n"

In [ ]:
print(prefixes)
print(scheme)
print(data)

In [ ]:
print(j)

['KANIA', 'M.H. BENCH: KANIA', 'M.H. VERMA', 'JAGDISH SARAN (J) RAMASWAMI', 'V. (J) II', 'Dr. ARIJIT PASAYAT', 'LOKESHWAR SINGH PANTA', 'P. SATHASIVAM', 'K. RAMASWAMY', 'S. SAGHIR AHMAD']


In [ ]:
a=[1,2]
a.append(3)
print(a)

[1, 2, 3]


In [ ]:
number_of_cases=0
case_end=0
judges=[]
p=[]
r=[]
j=[]
petetioner=''
respondent=''
data=""
casename=""
p_and_r_and_j =0
for line in lines:
  if '.txt' in line:
    number_of_cases+=1
    if case_end+1 != number_of_cases:
      p_index= len(p)
      if petetioner in p:
        p_index=p.index(petetioner)
      else:
        p.append(petetioner)
      
      r_index= len(r)
      if respondent in r:
        r_index=r.index(respondent)
      else:
        r.append(respondent)
      data+="dc:hasParty :P"+str(p_index+1)+", :R"+str(r_index+1)+";\n"


      j_index= []
      for i in judges:
        
        if i in j:
          j_index.append(j.index(i))
        else:
          j_index.append(len(j))
          j.append(i)
      print(j_index)   
      text=[]
      temp=''
      
      if len(judges)!=0:
        for i in j_index:
          text.append(":J"+str(i+1))
        for i in text:
          temp+=i+","
        temp=temp[:-1]

        data+="dc:hasCourtOfficial "+temp+";\n"
      data+=".\n"
      #data+=":case"+str(number_of_cases-1)+'P1 a schema:Petitioner, foaf:Person ;\n \trdfs:hasName "'+petetioner+'";\n .\n'
      #data+=":case"+str(number_of_cases-1)+'P1 a schema:Respondent, foaf:Person ;\n \trdfs:hasName "'+respondent+'";\n .\n'

      case_end+=1
      p_and_r_and_j =0
      judges=[]
      petetioner=''
      respondent=''
    data+=":case"+str(number_of_cases)+"\n"

  if 'hasCaseName' in line:
    casename= line.split("hasCaseName ",1)[1]
    temp= casename.replace(" ", "_")
    temp=re.sub('[^A-Za-z0-9_]+', '', temp)
    data+="a schema:"+str(temp.strip())+";\n"
    data+='rdfs:label "'+casename.strip()+'";\n'

  if 'hasDate' in line:#to be done using loop 
    tmp= line.split("hasDate ",1)[1]
    data+='rdfs:hasDate "'+tmp.strip()+'";\n'

  if 'hasAppealNo' in line:
    tmp= line.split("hasAppealNo ",1)[1]
    data+='rdfs:hasAppealNo "'+tmp.strip()+'";\n'

  if 'hasCaseNo' in line:
    tmp= line.split("hasCaseNo ",1)[1]
    data+='rdfs:hasCaseNo "'+tmp.strip()+'";\n'
  
  if 'hasCaseType' in line:
    tmp= line.split("hasCaseType ",1)[1]
    data+='rdfs:hasCaseType "'+tmp.lower()+'";\n'
  
  if 'hasAuthor' in line:
    tmp= line.split("hasAuthor ",1)[1]
    data+='rdfs:hasAuthor "'+tmp.strip()+'";\n'
  
  if 'hasCourt' in line:
    tmp= line.split("hasCourt",1)[1]
    data+='rdfs:hasCourt "'+tmp.strip()+'";\n'
  
  if 'hasLocation' in line:
    tmp= line.split("hasLocation ",1)[1]
    data+='rdfs:hasLocation "'+tmp.strip()+'";\n'



  if 'hasParty PETITIONER' in line:
    p_and_r_and_j=1
  if 'hasParty RESPONDENT' in line:
    p_and_r_and_j=2
  if 'hasCourtOfficial Judge' in line:
    p_and_r_and_j=3

  if 'hasName' in line:
    tmp= line.split("hasName ",1)[1]
    if p_and_r_and_j==1:
      petetioner+=tmp.strip()
    if p_and_r_and_j==2:
      respondent+=tmp.strip()      
    if p_and_r_and_j==3:
      judges.append(tmp.strip())



p_index= len(p)
if petetioner in p:
  p_index=p.index(petetioner)
else:
  p.append(petetioner)

r_index= len(r)
if respondent in r:
  r_index=r.index(respondent)
else:
  r.append(respondent)
data+="dc:hasParty :P"+str(p_index+1)+", :R"+str(r_index+1)+";\n"


j_index=[]
for i in judges:
  
  print(i)
  if i in j:
    j_index.append(j.index(i))
  else:
    print(2)
    t=len(j)
    j_index.append(t)
    
    j.append(i)
print(j_index)
text=[]
temp=''

if len(judges)!=0:
  for i in j_index:
    print(i)
    text.append(":J"+str(i+1))
  for i in text:
    temp+=i+","
  temp=temp[:-1]

  data+="dc:hasCourtOfficial "+temp+";\n"
data+=".\n"

  


In [ ]:
c='arfknf'
d='afhiflzk'
for i,j in c,d:
  print(i,j)

ValueError: ignored

In [ ]:
import pandas as pd 
    
# making dataframe 
df = pd.read_csv("Judges.csv") 
   
# output the dataframe
print(df)
judges_name = df['Name'].tolist()

judges_name
judges_name = df['Name'].tolist()
judges_name=[e[21:] for e in judges_name]
print(judges_name)
